# Add upstream, downstream and basin information to the dataframe

* Purpose of script: add contextual data to the datafram. 
* Author: Rutger Hofste
* Kernel used: python35
* Date created: 20170915

In [1]:
import time
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
print(dateString,timeString)

Y2017M09D28 UTC 13:19


In [2]:
S3_INPUT_PATH_EE  = "s3://wri-projects/Aqueduct30/processData/Y2017M09D14_RH_merge_EE_results_V01/output/"

S3_INPUT_PATH_HYDROBASINS = "s3://wri-projects/Aqueduct30/processData/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/"

EC2_INPUT_PATH = "/volumes/data/Y2017M09D15_RH_Add_Basin_Data_V01/input"
EC2_OUTPUT_PATH = "/volumes/data/Y2017M09D15_RH_Add_Basin_Data_V01/output"

INPUT_FILENAME_EE =  "mergedZonalStatsEE_V12.pkl"
INPUT_FILENAME_HYDROBASINS =  "hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01.pkl"


In [3]:
!rm -r {EC2_INPUT_PATH} 
!rm -r {EC2_OUTPUT_PATH} 

In [4]:
!mkdir -p {EC2_INPUT_PATH} 
!mkdir -p {EC2_OUTPUT_PATH} 

In [5]:
!aws s3 cp {S3_INPUT_PATH_EE} {EC2_INPUT_PATH} --recursive

download: s3://wri-projects/Aqueduct30/processData/Y2017M09D14_RH_merge_EE_results_V01/output/mergedZonalStatsEE_V12.pkl to ../../../../data/Y2017M09D15_RH_Add_Basin_Data_V01/input/mergedZonalStatsEE_V12.pkl
download: s3://wri-projects/Aqueduct30/processData/Y2017M09D14_RH_merge_EE_results_V01/output/mergedZonalStatsEE_V12.csv to ../../../../data/Y2017M09D15_RH_Add_Basin_Data_V01/input/mergedZonalStatsEE_V12.csv


In [6]:
!aws s3 cp {S3_INPUT_PATH_HYDROBASINS} {EC2_INPUT_PATH} --recursive --exclude "*" --include "*.pkl"

download: s3://wri-projects/Aqueduct30/processData/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01.pkl to ../../../../data/Y2017M09D15_RH_Add_Basin_Data_V01/input/hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01.pkl


In [7]:
import os
import pandas as pd
import multiprocessing as mp
import pickle
import numpy as np
import itertools
import logging
import pprint
import ast

In [8]:
inputLocationEE = os.path.join(EC2_INPUT_PATH,INPUT_FILENAME_EE)
inputLocationHydroBasins = os.path.join(EC2_INPUT_PATH,INPUT_FILENAME_HYDROBASINS)

In [9]:
df_ee = pd.read_pickle(inputLocationEE)

In [10]:
df_ee.index.names = ['PFAF_ID']

In [11]:
df_HydroBasins = pd.read_pickle(inputLocationHydroBasins)


In [12]:
df_complete = df_HydroBasins.merge(df_ee,how="left",left_index=True, right_index=True)

In [13]:
df_complete.dtypes

HYBAS_ID2                                   int64
Unnamed: 0                                  int64
HYBAS_ID                                    int64
NEXT_DOWN                                   int64
NEXT_SINK                                   int64
MAIN_BAS                                    int64
DIST_SINK                                 float64
DIST_MAIN                                 float64
SUB_AREA                                  float64
UP_AREA                                   float64
PFAF_ID                                     int64
ENDO                                        int64
COAST                                       int64
ORDER                                       int64
SORT                                        int64
Upstream_HYBAS_IDs                         object
Upstream_PFAF_IDs                          object
Downstream_HYBAS_IDs                       object
Downstream_PFAF_IDs                        object
NEXT_SINK_PFAF                            float64


## Functions

In [65]:
def calculateTotalDemand(useType,temporalResolution,year,month):
    # This function will add Dom Ind IrrLinear and Livestock of all basins in the input list
    
    if temporalResolution == "year":
        keyTotal = "local_sum_volumem3_Tot%s_%s_Y%0.4d" %(useType,temporalResolution,year)
    else:
        keyTotal = "local_sum_volumem3_Tot%s_%s_Y%0.4dM%0.2d" %(useType,temporalResolution,year,month)
    
    # Create Column with zeros
    dfDemand[keyTotal] = 0
    for demandType in demandTypes:
        if demandType == "IrrLinear" and temporalResolution == "year":
            key = "total_volume_%s%s_%sY%0.4d" %(demandType,useType,temporalResolution,year)
        else:
            key = "total_volume_%s%s_%sY%0.4dM%0.2d" %(demandType,useType,temporalResolution,year,month)
        dfDemand[keyTotal] = dfDemand[keyTotal] + df_complete[key]
    return dfDemand   


def calculateUpstream():
    # This function will add upstream data to the dataFrame 
    # standard column format: upstream_sum_volumem3_TotWW_monthY2014M12
    pass

dfTest = pd.DataFrame()
 
def addUpstream2(listje):
    df_full_temp = df_complete.copy()
    df_part_temp = df_full_temp[df_full_temp.index.isin(listje)]
    df_part_temp2 = df_part_temp.copy()
    df_out = df_part_temp2.copy()
    
    i = 0
    for index, row in df_part_temp2.iterrows():
        i += 1
        print("i: ",i  ," index: ", index)
        upstreamCatchments = df_part_temp2.loc[index, "Upstream_PFAF_IDs"]
        upstreamCatchments = ast.literal_eval(upstreamCatchments)
        df_upstream = df_full_temp[df_full_temp.index.isin(upstreamCatchments)]
        # selecting columns based on regular expression
        df_upstream = df_upstream.filter(regex=("total*"))
        df_upstream = df_upstream.add_prefix("upstream_")
        sumSeries = df_upstream.sum(0)
        for key, value in sumSeries.iteritems():
            df_out.loc[index, key] = value
        df_out.loc[index, "errorCode"] = 0
        
        
    
    return(df_upstream,df_out)
        
    
    
def addUpstream(listje):
    df_full_temp = df_complete.copy()
    df_part_temp = df_full_temp[df_full_temp.index.isin(listje)]
    df_part_temp2 = df_part_temp.copy()
    df_out = df_part_temp2.copy()
    i = 0
    for index, row in df_part_temp2.iterrows():
        i += 1
        print("i: ",i  ," index: ", index)
        try:
            upstreamCatchments = df_part_temp2.loc[index, "Upstream_PFAF_IDs"]
            upstreamCatchments = ast.literal_eval(upstreamCatchments)
            df_upstream = df_full_temp.loc[upstreamCatchments]
            area = df_upstream["countarea30sm2"] * df_upstream["meanarea30sm2"]

            df_new = pd.DataFrame()
            df_new["aream2"] = area

            for parameter in parameterList:
                df_new["count_" + parameter] = df_upstream["count" + parameter]
                df_new["volumem3_" + parameter] = area * df_upstream["mean" + parameter]

            sumSeries = df_new.sum()

            for key, value in sumSeries.iteritems():
                newKey = "upstream_sum_" + key
                df_out.loc[index, newKey] = value
            df_out.loc[index, "errorCode"] = 0
        except:
            print("error")
            df_out.loc[index, "errorCode"] = 1
            pass

    return df_out   
    
    

## Script

In [18]:
demandTypes = ["PDom","PInd","IrrLinear","PLiv"]
useTypes = ["WW","WN"]
temporalResolutions = ["year","month"]
years = [2014]

In [19]:
dfDemand = pd.DataFrame(index=df_complete.index)
for temporalResolution in temporalResolutions:
    for useType in useTypes:
        for year in years:
            if temporalResolution == "year":
                month = 12
                print(useType,temporalResolution,year,month)
                dfDemand = calculateTotalDemand(useType,temporalResolution,year,month)
            else:
                for month in range(1,13):
                    print(useType,temporalResolution,year,month)
                    dfDemand = calculateTotalDemand(useType,temporalResolution,year,month)          

WW year 2014 12
WN year 2014 12
WW month 2014 1
WW month 2014 2
WW month 2014 3
WW month 2014 4
WW month 2014 5
WW month 2014 6
WW month 2014 7
WW month 2014 8
WW month 2014 9
WW month 2014 10
WW month 2014 11
WW month 2014 12
WN month 2014 1
WN month 2014 2
WN month 2014 3
WN month 2014 4
WN month 2014 5
WN month 2014 6
WN month 2014 7
WN month 2014 8
WN month 2014 9
WN month 2014 10
WN month 2014 11
WN month 2014 12


In [20]:
dfDemand.head()

,local_sum_volumem3_TotWW_year_Y2014,local_sum_volumem3_TotWN_year_Y2014,local_sum_volumem3_TotWW_month_Y2014M01,local_sum_volumem3_TotWW_month_Y2014M02,local_sum_volumem3_TotWW_month_Y2014M03,local_sum_volumem3_TotWW_month_Y2014M04,local_sum_volumem3_TotWW_month_Y2014M05,local_sum_volumem3_TotWW_month_Y2014M06,local_sum_volumem3_TotWW_month_Y2014M07,local_sum_volumem3_TotWW_month_Y2014M08,...,local_sum_volumem3_TotWN_month_Y2014M03,local_sum_volumem3_TotWN_month_Y2014M04,local_sum_volumem3_TotWN_month_Y2014M05,local_sum_volumem3_TotWN_month_Y2014M06,local_sum_volumem3_TotWN_month_Y2014M07,local_sum_volumem3_TotWN_month_Y2014M08,local_sum_volumem3_TotWN_month_Y2014M09,local_sum_volumem3_TotWN_month_Y2014M10,local_sum_volumem3_TotWN_month_Y2014M11,local_sum_volumem3_TotWN_month_Y2014M12
PFAF_ID,,,,,,,,,,,,,,,,,,,,,
111011,6.257741e+07,2.915234e+07,5.071952e+06,5.039243e+06,5.096070e+06,5.268799e+06,5.233163e+06,5.282180e+06,5.382082e+06,5.472195e+06,...,2.372698e+06,2.448676e+06,2.443976e+06,2.468962e+06,2.509266e+06,2.543447e+06,2.498844e+06,2.432985e+06,2.380002e+06,2.353863e+06
111012,8.204290e+06,3.388488e+06,6.827566e+05,6.829166e+05,6.831846e+05,6.836583e+05,6.840569e+05,6.843250e+05,6.844042e+05,6.844215e+05,...,2.818678e+05,2.823415e+05,2.827401e+05,2.830082e+05,2.830874e+05,2.831047e+05,2.829387e+05,2.826311e+05,2.820958e+05,2.816334e+05
111013,7.569537e+06,3.111568e+06,6.301277e+05,6.302355e+05,6.304261e+05,6.307669e+05,6.310496e+05,6.312474e+05,6.313145e+05,6.313229e+05,...,2.589287e+05,2.592695e+05,2.595522e+05,2.597500e+05,2.598171e+05,2.598255e+05,2.597075e+05,2.594838e+05,2.591036e+05,2.587620e+05
111014,1.046668e+06,5.024309e+05,8.651350e+04,8.664306e+04,8.685163e+04,8.721648e+04,8.751861e+04,8.770851e+04,8.775453e+04,8.777285e+04,...,4.149858e+04,4.186342e+04,4.216556e+04,4.235545e+04,4.240148e+04,4.241980e+04,4.227204e+04,4.204755e+04,4.165468e+04,4.130191e+04
111015,1.118431e+08,4.668647e+07,9.301079e+06,9.303778e+06,9.309980e+06,9.319181e+06,9.327492e+06,9.333573e+06,9.335649e+06,9.335846e+06,...,3.881478e+06,3.889610e+06,3.896934e+06,3.902287e+06,3.904112e+06,3.904281e+06,3.900583e+06,3.894476e+06,3.886032e+06,3.877061e+06


In [21]:
df_smaller = 

In [30]:
listje = [292107]

In [62]:
df_upstream_sum,df_out = addUpstream2(listje)

In [63]:
df_upstream_sum

,upstream_sum_total_area_30s_m2,upstream_sum_total_volume_IrrLinearWN_monthY2014M01,upstream_sum_total_volume_IrrLinearWN_monthY2014M04,upstream_sum_total_volume_PLivWN_monthY2014M03,upstream_sum_total_volume_runoff_yearY2014M12,upstream_sum_total_volume_IrrLinearWW_monthY2014M06,upstream_sum_total_volume_reducedmeanrunoff_month_Y1960Y2014M8,upstream_sum_total_volume_PIrrWW_monthY2014M04,upstream_sum_total_volume_IrrLinearWW_monthY2014M09,upstream_sum_total_volume_PIrrWW_monthY2014M11,...,upstream_sum_total_volume_PIrrWN_monthY2014M10,upstream_sum_total_volume_runoff_monthY2014M02,upstream_sum_total_volume_PLivWN_monthY2014M10,upstream_sum_total_volume_PDomWN_yearY2014M12,upstream_sum_total_volume_runoff_monthY2014M10,upstream_sum_total_volume_PIrrWW_monthY2014M08,upstream_sum_total_volume_IrrLinearWW_monthY2014M04,upstream_sum_total_volume_PLivWW_yearY2014M12,upstream_sum_total_volume_PIrrWN_monthY2014M07,upstream_sum_total_volume_IrrLinearWN_monthY2014M05
PFAF_ID,,,,,,,,,,,,,,,,,,,,,
292108,3.591820e+09,0.000000e+00,0.000000e+00,6776.336645,8.918357e+07,0.000000e+00,6.073141e+06,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,2.086731e+07,7167.884710,2.621435e+06,1.402903e+05,0.000000e+00,0.000000e+00,85100.494113,0.000000e+00,0.000000e+00
292109,1.808558e+10,7.386851e+06,8.708965e+06,64981.950872,1.227492e+09,1.035372e+08,1.015445e+08,1.239052e+07,7.241379e+07,3.115129e+07,...,1.966965e+07,3.884545e+08,68148.543201,2.117459e+07,1.556081e+07,1.082188e+08,2.303959e+07,813498.382533,4.415910e+07,2.721679e+07


In [49]:
test = df_upstream_sum.sum(0)

In [64]:
df_out.head()

,HYBAS_ID2,Unnamed: 0,HYBAS_ID,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,...,upstream_sum_total_volume_PIrrWN_monthY2014M10,upstream_sum_total_volume_runoff_monthY2014M02,upstream_sum_total_volume_PLivWN_monthY2014M10,upstream_sum_total_volume_PDomWN_yearY2014M12,upstream_sum_total_volume_runoff_monthY2014M10,upstream_sum_total_volume_PIrrWW_monthY2014M08,upstream_sum_total_volume_IrrLinearWW_monthY2014M04,upstream_sum_total_volume_PLivWW_yearY2014M12,upstream_sum_total_volume_PIrrWN_monthY2014M07,upstream_sum_total_volume_IrrLinearWN_monthY2014M05
PFAF_ID,,,,,,,,,,,,,,,,,,,,,
292107,2060877600,6355,2060877600,2060879580,2060085630,2060085630,19.0,19.0,155.4,21859.5,...,1.966965e+07,4.093218e+08,75316.427911,2.379602e+07,1.570110e+07,1.082188e+08,2.303959e+07,898598.876645,4.415910e+07,2.721679e+07


In [ ]:
mp.cpu_count()

In [ ]:
print(inputLocation)

In [ ]:
indices_full = df_full.index.values
indices_split = np.array_split(indices_full, mp.cpu_count())

In [ ]:
print(indices_split)

In [ ]:
mp.log_to_stderr()

In [ ]:
logger = mp.get_logger()
logger.setLevel(logging.INFO)

In [ ]:
pool = mp.Pool(mp.cpu_count())

In [ ]:
df_out = pd.concat(pool.map(addUpstream, indices_split))

In [ ]:
df_out = addUpstream([1])

In [ ]:
df_out.head()